In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def  __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # affine: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(net.parameters())
# print(params)
print(len(params))
print(params[0].size())

[Parameter containing:
tensor([[[[ 0.1576,  0.0654,  0.0913,  0.1753, -0.1292],
          [-0.0004, -0.1088, -0.1886,  0.1094, -0.0192],
          [-0.0311, -0.1011, -0.1485,  0.0566, -0.0025],
          [ 0.0004,  0.1624, -0.0279,  0.0794,  0.1859],
          [-0.0462, -0.0324,  0.0207,  0.1279, -0.0446]]],


        [[[-0.0653, -0.0064,  0.0155, -0.1185, -0.1307],
          [-0.0826, -0.1754, -0.1637,  0.0091, -0.0694],
          [-0.1310, -0.0415, -0.1741, -0.1643,  0.1941],
          [-0.0860, -0.1352,  0.0092,  0.0617, -0.0982],
          [-0.0959,  0.1060, -0.0735,  0.0776,  0.1806]]],


        [[[-0.0773,  0.0842, -0.0534,  0.1830, -0.1916],
          [ 0.1940,  0.0005, -0.1535,  0.1873,  0.1704],
          [-0.1671,  0.1499, -0.0624,  0.0479,  0.1047],
          [ 0.1157,  0.0716, -0.1519, -0.0764, -0.1924],
          [-0.1991,  0.1698, -0.1676,  0.0430, -0.0726]]],


        [[[-0.0559, -0.1857,  0.1817, -0.1988,  0.1558],
          [ 0.1765,  0.1676,  0.0078,  0.0143,  0.077

In [4]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0276, -0.0644,  0.1242, -0.0539,  0.0229,  0.0126, -0.0931, -0.0868,
          0.0859,  0.0856]], grad_fn=<AddmmBackward0>)


In [5]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [6]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.3254, grad_fn=<MseLossBackward0>)


In [7]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [9]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0062,  0.0091, -0.0085, -0.0071,  0.0035,  0.0254])


In [ ]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [ ]:
import torch.optim as optim

# Optimizer를 생성
optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()   # 변화도 버퍼를 0으로
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()